<a href="https://colab.research.google.com/github/tuhinkm/MAS-Agentic-AI/blob/main/Strmlit_UI_Gemini_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objective
We shall the the functions from Optimization Agentic Tools - Comparison of Gemini and OpenAI.ipynb file and convert into a strmlit project

In [1]:
!pip install langchain_community --quiet
!pip install langchain_groq --quiet
!pip install langchain-google-genai --quiet
!pip install langchain-openai --quiet
!apt-get install -y -qq coinor-cbc --quiet
!pip install pyomo --quiet
!pip install streamlit --quiet

In [2]:
%%writefile app.py
from langchain.prompts import PromptTemplate
from langchain_core.tools import tool
import langchain_community
import warnings
warnings.filterwarnings('ignore')
import math
import os
from geopy.geocoders import Nominatim
from langchain_core.messages import AIMessage,HumanMessage
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from google.colab import userdata
from langchain.agents import tool
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from pyomo.environ import *
import pyomo
import random
import numpy as np
import streamlit as st

# input dictionary
input_dict = {}
random.seed(33)
for i in range(1,25):
  input_dict['hour_'+str(i)]= random.randint(23,79)

# Model setting
os.environ["GOOGLE_API_KEY"] = userdata.get('GEMINI_TOKEN')
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

def optimizer_run(human_query, input_dict, llm):
    """
    This function simulates your agent's response. This function takes another two functions and perform
    calculations intended by the users
    """

    @tool
    def get_dict_values(input_dict:dict, high_percentile:int, low_percentile:int=10):
      '''
      This function takes dictionary as input and return defined percentile ranges value of a dictionary
      '''
      # Extract the values from the dictionary
      values = list(input_dict.values())

      # Calculate the 75th percentile
      high_val = np.percentile(values, high_percentile)
      low_val = np.percentile(values, low_percentile)

      return (high_val, low_val)

    @tool
    def sample_optimization(high_val:float, low_val:float):
      '''
      This is an optimization function which will take two integer inputs, perform the necessary
      calculations and provide float output
      '''
      model = ConcreteModel()

      # Define variables
      model.x = Var(within=NonNegativeReals)
      model.y = Var(within=NonNegativeReals)

      # Define objective function: Minimize x + y
      model.objective = Objective(expr=model.x + model.y, sense=minimize)

      # Define constraints
      model.constraint1 = Constraint(expr=model.x + 2*model.y >= int(high_val))
      model.constraint2 = Constraint(expr=model.x - model.y <= int(low_val))

      # Solve the model using GLPK solver
      solver = pyomo.environ.SolverFactory('cbc')
      # solver = pyomo.environ.SolverFactory('gurobi')
      solver.solve(model)

      return (value(model.objective))

    prompt_gemini = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an agentic AI solution. You have 2 functions which you need to execute as per human input prompt. The first function is 'get_dict_values' function which will take an input dictionary and then two integer values from human prompt, which are called high percentile value and low percentile value. Use the dictionary provided in the 'input_dict' variable for this function. In case any of the two numbers are input anything other than integer, please return a message as 'You should provide only integer values as high percentile and low percentile values. Please modify the prompt' and complete the chain. For valid values of high percentile and low percentile, execute 'get_dict_values' and generate high_val and low_val parameters. Use these parameters as input to 'sample_optimization' function and generate the final output."),
        ("human", "{input}\n\nHere is the dictionary to use:\n{input_dict}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
    )

    tools = [get_dict_values, sample_optimization]
    agent = create_tool_calling_agent(llm, tools, prompt_gemini)

    # run
    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

    result = agent_executor.invoke({"input":  human_query, "input_dict": input_dict})
    return(result["output"])

st.title("Streamlit Chatbot with Agent Integration")

# Initialize chat history
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display chat messages from history on app rerun
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# Accept user input
if prompt := st.chat_input("Ask me anything..."):
    # Add user message to chat history
    st.session_state.messages.append({"role": "user", "content": prompt})
    # Display user message in chat message container
    with st.chat_message("user"):
        st.markdown(prompt)

    # Get agent's response
    agent_response = optimizer_run(prompt, input_dict, llm)

    # Add agent's response to chat history
    st.session_state.messages.append({"role": "assistant", "content": agent_response})
    # Display agent's response in chat message container
    with st.chat_message("assistant"):
        st.markdown(agent_response)

Overwriting app.py


## Install localtunnel

In [3]:
!npm install localtunnel
!npm audit fix --force

⠙⠹⠸⠼⠴⠦
up to date, audited 23 packages in 890ms
⠦
⠦3 packages are looking for funding
⠦  run `npm fund` for details
⠦
2 high severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠦npm warn using --force Recommended protections disabled.
⠙⠹⠸⠼npm warn audit Updating localtunnel to 1.8.3, which is a SemVer major change.
⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏npm warn deprecated cryptiles@2.0.5: This version has been deprecated in accordance with the hapi support policy (hapi.im/support). Please upgrade to the latest version to get the best features, bug fixes, and security patches. If you are unable to upgrade at this time, paid support is available for older versions (hapi.im/commercial).
⠏npm warn deprecated boom@2.10.1: This version has been deprecated in accordance with the hapi support policy (hapi.im/support). Please upgrade to the latest version to get the best fe

## Run streamlit in background

In [4]:
!streamlit run /content/app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.63.116.10
⠙⠹your url is: https://gold-points-prove.loca.lt
